In [1]:
#! ./install_local_setup.sh
! cp ../col-de-port_summa_setup_fresh/params/local_param_info.txt ./params/local_param_info.txt
! cp ../col-de-port_summa_setup_fresh/params/local_attributes.nc ./params/local_attributes.nc
! cp ../col-de-port_summa_setup_fresh/params/parameter_trial.nc ./params/parameter_trial.nc
! cp ../col-de-port_summa_setup_fresh/settings/decisions.txt ./settings/decisions.txt
! cp ../col-de-port_summa_setup_fresh/settings/output_control.txt ./settings/output_control.txt

In [2]:
%pylab inline
import pandas as pd
import pysumma as ps
import xarray as xr
from matplotlib import cm
import seaborn as sns
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
sns.set_context('talk')
mpl.style.use('seaborn-bright')
mpl.rcParams['figure.figsize'] = (18, 12)

Populating the interactive namespace from numpy and matplotlib


In [22]:
def validate_layer_params(params):
    for i in range(1,5):
        assert params[f'zmaxLayer{i}_upper'] <= params[f'zmaxLayer{i}_lower'], \
               "{} - {} - {}".format(i, params[f'zmaxLayer{i}_upper'], params[f'zmaxLayer{i}_lower'])
        assert params[f'zmaxLayer{i}_upper'] / params[f'zminLayer{i}'] >= 2.5, \
               "{} - {} - {}".format(i, params[f'zmaxLayer{i}_upper'], params[f'zminLayer{i}'])
        assert params[f'zmaxLayer{i}_upper'] / params[f'zminLayer{i+1}'] >= 2.5, \
               "{} - {} - {}".format(i, params[f'zmaxLayer{i}_upper'], params[f'zminLayer{i+1}'])

In [23]:
default_params = {'zminLayer1':      0.0075,   'zminLayer2':       0.0100,  'zminLayer3':       0.0500,  'zminLayer4':       0.1000,  'zminLayer5': 0.2500, 
                  'zmaxLayer1_lower': 0.0500,  'zmaxLayer2_lower': 0.2000,  'zmaxLayer3_lower': 0.5000,  'zmaxLayer4_lower': 1.0000, 
                  'zmaxLayer1_upper': 0.0300,  'zmaxLayer2_upper': 0.1500,  'zmaxLayer3_upper': 0.3000,  'zmaxLayer4_upper': 0.7500 }

one_layer     = {'zminLayer1':       0.0075, 'zminLayer2':       1000.0, 'zminLayer3':       1000., 'zminLayer4':       1000., 'zminLayer5': 1000.,
                 'zmaxLayer1_lower': 9000.0, 'zmaxLayer2_lower': 9000.0, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000., 
                 'zmaxLayer1_upper': 3000.0, 'zmaxLayer2_upper': 9000.0, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000. } 

thick_2_layer = {'zminLayer1':       0.0075, 'zminLayer2':       0.0100, 'zminLayer3':        100., 'zminLayer4':      100., 'zminLayer5': 100.,
                 'zmaxLayer1_lower': 0.1250, 'zmaxLayer2_lower': 9000.0, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000., 
                 'zmaxLayer1_upper': 0.1250, 'zmaxLayer2_upper': 9000.0, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000. } 

mid_2_layer = {'zminLayer1':       0.0075, 'zminLayer2':       0.0100, 'zminLayer3':        100., 'zminLayer4':      100., 'zminLayer5': 100.,
               'zmaxLayer1_lower': 0.050, 'zmaxLayer2_lower': 9000.0, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000., 
               'zmaxLayer1_upper': 0.050, 'zmaxLayer2_upper': 9000.0, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000. } 

thin_2_layer = {'zminLayer1':       0.0075, 'zminLayer2':       0.0100, 'zminLayer3':        100., 'zminLayer4':      100., 'zminLayer5': 100.,
                'zmaxLayer1_lower': 0.0250, 'zmaxLayer2_lower': 9000.0, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000., 
                'zmaxLayer1_upper': 0.0250, 'zmaxLayer2_upper': 9000.0, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000. } 

thick_3_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 1000., 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.125, 'zmaxLayer2_lower': 0.2, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.125, 'zmaxLayer2_upper': 0.2, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000.}
    
mid_3_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 1000., 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.050, 'zmaxLayer2_lower': 0.2, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.050, 'zmaxLayer2_upper': 0.2, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000.}

thin_3_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 1000., 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.025, 'zmaxLayer2_lower': 0.2, 'zmaxLayer3_lower': 9000., 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.025, 'zmaxLayer2_upper': 0.2, 'zmaxLayer3_upper': 9000., 'zmaxLayer4_upper': 9000.}

thick_4_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 0.015, 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.075, 'zmaxLayer2_lower': 0.15, 'zmaxLayer3_lower': 0.20, 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.075, 'zmaxLayer2_upper': 0.15, 'zmaxLayer3_upper': 0.20, 'zmaxLayer4_upper': 9000.}
    
mid_4_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 0.015, 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.050, 'zmaxLayer2_lower': 0.125, 'zmaxLayer3_lower': 0.20, 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.050, 'zmaxLayer2_upper': 0.125, 'zmaxLayer3_upper': 0.20, 'zmaxLayer4_upper': 9000.}

thin_4_layer = {'zminLayer1': 0.01, 'zminLayer2': 0.01, 'zminLayer3': 0.01, 'zminLayer4': 0.015, 'zminLayer5': 1000.,
               'zmaxLayer1_lower': 0.025, 'zmaxLayer2_lower': 0.100, 'zmaxLayer3_lower': 0.20, 'zmaxLayer4_lower': 9000.,
               'zmaxLayer1_upper': 0.025, 'zmaxLayer2_upper': 0.100, 'zmaxLayer3_upper': 0.20, 'zmaxLayer4_upper': 9000.}

for i, l in enumerate([one_layer, thin_2_layer, thick_2_layer, mid_2_layer, 
                       thick_3_layer, mid_3_layer, thin_3_layer,
                       thick_4_layer, mid_4_layer, thin_4_layer]):
    print(i)
    validate_layer_params(l)
    

config_all = {'{}K_CLM'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                     'decisions': {'snowLayers': 'CLM_2010'},
                                     'parameters': default_params} for dt in dt_list}

config_1layer = {'{}K_1L_all'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                            'decisions': {'snowLayers': 'CLM_2010'},
                                            'parameters': one_layer} for dt in dt_list}

config_2layer = {'{}K_2L_thin'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                            'decisions': {'snowLayers': 'CLM_2010'},
                                            'parameters': thin_2_layer} for dt in dt_list}
config_2layer.update({'{}K_2L_mid'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                 'decisions': {'snowLayers': 'CLM_2010'},
                                                 'parameters': mid_2_layer} for dt in dt_list})
config_2layer.update({'{}K_2L_thick'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                   'decisions': {'snowLayers': 'CLM_2010'},
                                                   'parameters': thick_2_layer} for dt in dt_list})

config_3layer = {'{}K_3L_thin'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                            'decisions': {'snowLayers': 'CLM_2010'},
                                            'parameters': thin_3_layer} for dt in dt_list}
config_3layer.update({'{}K_3L_mid'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                 'decisions': {'snowLayers': 'CLM_2010'},
                                                 'parameters': mid_3_layer} for dt in dt_list})
config_3layer.update({'{}K_3L_thick'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                   'decisions': {'snowLayers': 'CLM_2010'},
                                                   'parameters': thick_3_layer} for dt in dt_list})

config_4layer = {'{}K_4L_thin'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                            'decisions': {'snowLayers': 'CLM_2010'},
                                            'parameters': thin_4_layer} for dt in dt_list}
config_4layer.update({'{}K_4L_mid'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                 'decisions': {'snowLayers': 'CLM_2010'},
                                                 'parameters': mid_4_layer} for dt in dt_list})
config_4layer.update({'{}K_4L_thick'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                                   'decisions': {'snowLayers': 'CLM_2010'},
                                                   'parameters': thick_4_layer} for dt in dt_list})

config_jrdn = {'{}K_JRDN'.format(dt): {'file_manager': './file_manager_{}K.txt'.format(dt),
                                       'decisions': {'snowLayers': 'jrdn1991'},
                                       'parameters': default_params} for dt in dt_list}

0
1
2
3
4
5
6
7
8
9


In [24]:
config_all.update(config_jrdn)
config_all.update(config_2layer)
config_all.update(config_3layer)
config_all.update(config_4layer)

In [25]:
len(list(config_all.keys()))

44

In [26]:
#failed = ['+4.0K_CLM', '+4.0K_3L_thick']
#config_all = {k: config_all[k] for k in failed}

In [27]:
executable = '/pool0/data/andrbenn/summa/bin/summa.exe'

In [28]:
ens_dt = ps.Ensemble(executable, config_all, num_workers=44)

In [29]:
ens_dt.run('local')

In [30]:
print(ens_dt.summary()['error'])

[]


In [37]:
print(ens_dt.simulations['-2.0K_3L_thin'].stdout[-500:])

  0
2006  9 16  0  0
2006  9 17  0  0
2006  9 18  0  0
2006  9 19  0  0
2006  9 20  0  0
2006  9 21  0  0
2006  9 22  0  0
2006  9 23  0  0
2006  9 24  0  0
2006  9 25  0  0
2006  9 26  0  0
2006  9 27  0  0
2006  9 28  0  0
2006  9 29  0  0
2006  9 30  0  0
2006 10  1  0  0
2006 10  2  0  0
2006 10  3  0  0
           2
 dtSave, dt_sub   14.062500000000000        7.0312500000000000     


FATAL ERROR: run_oneGRU/run_oneHRU/coupled_em/length of the coupled step is below the minimum step length




In [ ]:
ens_dt.rerun_failed('local')

In [ ]:
print(ens_dt.summary()['error'])